# 🤗 x 🦾: Training SmolVLA with LeRobot Notebook

Welcome to the **LeRobot SmolVLA training notebook**! This notebook provides a ready-to-run setup for training imitation learning policies using the [🤗 LeRobot](https://github.com/huggingface/lerobot) library.

In this example, we train an `SmolVLA` policy using a dataset hosted on the [Hugging Face Hub](https://huggingface.co/), and optionally track training metrics with [Weights & Biases (wandb)](https://wandb.ai/).

## ⚙️ Requirements
- A Hugging Face dataset repo ID containing your training data (`--dataset.repo_id=YOUR_USERNAME/YOUR_DATASET`)
- Optional: A [wandb](https://wandb.ai/) account if you want to enable training visualization
- Recommended: GPU runtime (e.g., NVIDIA A100) for faster training

## ⏱️ Expected Training Time
Training with the `SmolVLA` policy for 20,000 steps typically takes **about 5 hours on an NVIDIA A100** GPU. On less powerful GPUs or CPUs, training may take significantly longer!

## Example Output
Model checkpoints, logs, and training plots will be saved to the specified `--output_dir`. If `wandb` is enabled, progress will also be visualized in your wandb project dashboard.


## Install conda
This cell uses `condacolab` to bootstrap a full Conda environment inside Google Colab.


In [2]:
!pip install -q condacolab
import condacolab
condacolab.install()

✨🍰✨ Everything looks OK!


## Install LeRobot
This cell clones the `lerobot` repository from Hugging Face, installs FFmpeg (version 7.1.1), and installs the package in editable mode.


In [3]:
!git clone https://github.com/huggingface/lerobot.git
!conda install ffmpeg=7.1.1 -c conda-forge
!cd lerobot && pip install -e .

fatal: destination path 'lerobot' already exists and is not an empty directory.
Channels:
 - conda-forge
Platform: linux-64
Solving environment: - \ | / - done

## Package Plan ##

  environment location: /usr/local

  added / updated specs:
    - ffmpeg=7.1.1


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    alsa-lib-1.2.14            |       hb9d3cd8_0         553 KB  conda-forge
    aom-3.9.1                  |       hac33072_0         2.6 MB  conda-forge
    attr-2.5.2                 |       h39aace5_0          66 KB  conda-forge
    ca-certificates-2025.8.3   |       hbd8a1cb_0         151 KB  conda-forge
    cairo-1.18.4               |       h3394656_0         955 KB  conda-forge
    certifi-2025.8.3           |     pyhd8ed1ab_0         155 KB  conda-forge
    conda-24.11.3              |  py311h38be061_0         1.1 MB  conda-forge
    dav1d-1.2.1                |     

## Weights & Biases login
This cell logs you into Weights & Biases (wandb) to enable experiment tracking and logging.

In [4]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


## Install SmolVLA dependencies

In [5]:
!cd lerobot && pip install -e ".[smolvla]"

Obtaining file:///content/lerobot
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for lerobot (pyproject.toml) ... done
  Created wheel for lerobot: filename=lerobot-0.3.4-0.editable-py3-none-any.whl size=15707 sha256=c271b4083186f57104b5630d40ed1d82535584e381280726e4d468ebb3d48221
  Stored in directory: /tmp/pip-ephem-wheel-cache-gn5y8v_w/wheels/15/0d/02/b9c6ff1c78574dee99101ad231194b3425eb4cd784ce8c8338
Successfully built lerobot
  Attempting uninstall: lerobot
    Found existing installation: lerobot 0.3.4
    Uninstalling lerobot-0.3.4:
      Successfully uninstalled lerobot-0.3.4


## Start training SmolVLA with LeRobot

This cell runs the `train.py` script from the `lerobot` library to train a robot control policy.  

Make sure to adjust the following arguments to your setup:

1. `--dataset.repo_id=YOUR_HF_USERNAME/YOUR_DATASET`:  
   Replace this with the Hugging Face Hub repo ID where your dataset is stored, e.g., `pepijn223/il_gym0`.

2. `--batch_size=64`: means the model processes 64 training samples in parallel before doing one gradient update. Reduce this number if you have a GPU with low memory.

3. `--output_dir=outputs/train/...`:  
   Directory where training logs and model checkpoints will be saved.

4. `--job_name=...`:  
   A name for this training job, used for logging and Weights & Biases.

5. `--policy.device=cuda`:  
   Use `cuda` if training on an NVIDIA GPU. Use `mps` for Apple Silicon, or `cpu` if no GPU is available.

6. `--wandb.enable=true`:  
   Enables Weights & Biases for visualizing training progress. You must be logged in via `wandb login` before running this.

In [ ]:
!cd lerobot/src && python lerobot/scripts/train.py \
  --policy.path=lerobot/smolvla_base \
  --policy.repo_id=lerobot/smolvla_base \
  --dataset.repo_id=badwolf256/perfume_pick \
  --batch_size=128 \
  --steps=20000 \
  --output_dir=outputs/train/my_smolvla \
  --job_name=my_smolvla_training \
  --policy.device=cuda \
  --wandb.enable=false

INFO 2025-09-08 06:47:26 ils/utils.py:48 Cuda backend detected, using cuda.
WARNING 2025-09-08 06:47:26 /policies.py:80 Device 'None' is not available. Switching to 'cuda'.
INFO 2025-09-08 06:47:26 ts/train.py:111 {'batch_size': 128,
 'dataset': {'episodes': None,
             'image_transforms': {'enable': False,
                                  'max_num_transforms': 3,
                                  'random_order': False,
                                  'tfs': {'brightness': {'kwargs': {'brightness': [0.8,
                                                                                   1.2]},
                                                         'type': 'ColorJitter',
                                                         'weight': 1.0},
                                          'contrast': {'kwargs': {'contrast': [0.8,
                                                                               1.2]},
                                                       'type': 'Col

## Login into Hugging Face Hub
Now after training is done login into the Hugging Face hub and upload the last checkpoint

In [7]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): Traceback (most recent call last):
  File "/usr/local/lib/python3.11/getpass.py", line 77, in unix_getpass
    passwd = _raw_input(prompt, stream, input=input)
             ^^^^^^^^^^^^^^^^^^^^^

In [8]:
!huggingface-cli upload ${HF_USER}/my_smolvla \
  /content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model

⚠️  Warning: 'huggingface-cli upload' is deprecated. Use 'hf upload' instead.
Traceback (most recent call last):
  File "/usr/local/bin/huggingface-cli", line 8, in <module>
    sys.exit(main())
             ^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/huggingface_cli.py", line 61, in main
    service.run()
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/upload.py", line 211, in run
    print(self._upload())
          ^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/site-packages/huggingface_hub/commands/upload.py", line 271, in _upload
    raise FileNotFoundError(f"No such file or directory: '{self.local_path}'.")
FileNotFoundError: No such file or directory: '/content/lerobot/outputs/train/my_smolvla/checkpoints/last/pretrained_model'.
